In [1]:
#!pip install ortools
from ortools.sat.python import cp_model

In [66]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

#!pip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.opt import SolverStatus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools

In [67]:
import matplotlib.pyplot as plt
import itertools
import math
import numpy as np
from datetime import datetime, time, timedelta

In [68]:
def SolveModel(model, solver):
  # Solve model
  opt = SolverFactory(solver)
  result = opt.solve(model)

  if (result.solver.status == SolverStatus.ok) and \
      (result.solver.termination_condition == TerminationCondition.optimal):
      # Do something when the solution in optimal and feasible
      print('Solution is Optimal')
  elif (result.solver.termination_condition == TerminationCondition.infeasible):
      # Do something when model in infeasible
      print('Solution is Infeasible')
  else:
          # Something else is wrong
      print("Solver Status:",  result.solver.status)

  # Solve time
  print('Solve Time: ', result.solver.wallclock_time)

In [62]:
Flights = {
'AAL2561':  {'Target':39.5,"Dept":'WILEY'},
'DAL4703':	{'Target':38.25,"Dept":'WILEY'},
'DAL2684': {'Target': 37.0, 'Dept': 'WILEY'},
'AAL3441': {'Target': 36.0, 'Dept': 'CHASZ'},
'AAL0639': {'Target': 35.0, 'Dept': 'WILEY'},
'EGF1218': {'Target': 33.0, 'Dept': 'NELYN'},
'DAL2777': {'Target': 32.0, 'Dept': 'WILEY'},
'AAL9355': {'Target': 30.0, 'Dept': 'SMITH'},
'AAL7665': {'Target': 26.0, 'Dept': 'WILEY'},
'EGF9310': {'Target': 27.0, 'Dept': 'CHASZ'},
'DAL9416': {'Target': 24.0, 'Dept': 'PHILS'},
'AAL8912': {'Target': 23.0, 'Dept': 'WILEY'},
'DAL5519': {'Target': 21.0, 'Dept': 'NELYN'},
'EGF0862': {'Target': 20.0, 'Dept': 'WILEY'},
'SKW3013': {'Target': 18.0, 'Dept': 'NICKY'},
'EGF1050': {'Target': 17.0, 'Dept': 'WILEY'},
'AAL1180': {'Target': 15.0, 'Dept': 'NELYN'},
'KAL0043': {'Target': 14.0, 'Dept': 'WILEY'},
'AAL7799': {'Target': 12.0, 'Dept': 'WORTH'},
'DAL5919': {'Target': 11.0, 'Dept': 'WILEY'},
'AAL3074': {'Target': 8.0, 'Dept': 'SMITH'},
'EGF0512': {'Target': 9.0, 'Dept': 'CHASZ'},
'AAL8365': {'Target': 7.0, 'Dept': 'WILEY'},
'DAL2964': {'Target': 5.0, 'Dept': 'WHALT'},
'AAL2435': {'Target': 4.0, 'Dept': 'WILEY'},
'AAL3546': {'Target': 2.0, 'Dept': 'WORTH'},
'DAL5917': {'Target': 1.0, 'Dept': 'WILEY'},
'DAL2306': {'Target': 0.0, 'Dept': 'PHILS'}
 }

Dept_Fix = {
'NOBLR': {'Direction': 'N', 'Neighbor': ['NELYN'], 'Closed': 0},
'NELYN': {'Direction': 'N', 'Neighbor': ['NOBLR', 'NIKKY'], 'Closed': 0},
'NIKKY': {'Direction': 'N', 'Neighbor': ['NELYN', 'KNAVE'], 'Closed': 0},
'KNAVE': {'Direction': 'N', 'Neighbor': ['NIKKY', 'NEXAR'], 'Closed': 0},
'NEXAR': {'Direction': 'N', 'Neighbor': ['KNAVE'], 'Closed': 0},
'PHILS': {'Direction': 'S', 'Neighbor': ['SMITH'], 'Closed': 0},
'SMITH': {'Direction': 'S', 'Neighbor': ['PHILS', 'CHASZ'], 'Closed': 0},
'CHASZ': {'Direction': 'S', 'Neighbor': ['SMITH','ALISA'], 'Closed': 0},
'ALISA': {'Direction': 'S', 'Neighbor': ['CHASZ',  'EVANS'], 'Closed': 0},
'EVANS': {'Direction': 'S', 'Neighbor': ['ALISA'], 'Closed': 0},
'BEATY': {'Direction': 'E', 'Neighbor': ['EARTZ'], 'Closed': 0},
'EARTZ': {'Direction': 'E', 'Neighbor': ['BEATY', 'BEAMR'], 'Closed': 0},
'BEAMR': {'Direction': 'E', 'Neighbor': ['EARTZ', 'TREAT'], 'Closed': 0},
'TREAT': {'Direction': 'E', 'Neighbor': ['BEAMR', 'CREAK'], 'Closed': 0},
'CREAK': {'Direction': 'E', 'Neighbor': ['TREAT'], 'Closed': 0},
'WILEY': {'Direction': 'W', 'Neighbor': ['WICKR'], 'Closed': 0},
'WICKR': {'Direction': 'W', 'Neighbor': ['WILEY', 'WORTH'], 'Closed': 1},
'WORTH': {'Direction': 'W', 'Neighbor': ['WICKR', 'WHALT',], 'Closed': 0},
'WHALT': {'Direction': 'W', 'Neighbor': ['WORTH', 'WYMON'], 'Closed': 0},
'WYMON': {'Direction': 'W', 'Neighbor': ['WHALT'], 'Closed': 0}
 }

def timediff(x):
    a = datetime.strptime(x, '%H:%M:%S')
    b = datetime.strptime('18:00:00', '%H:%M:%S')
    return (a - b).total_seconds()/60

#Flights

In [91]:
model = ConcreteModel()

model.I = [i for i in Flights.keys()]

model.J = [i for i in Dept_Fix.keys()]

model.X = Var(model.I, model.J, bounds = (-10,10), within = Integers)
model.Y = Var(model.I, model.J, within = Boolean)

model.Closure = Param(model.J, initialize = {i: Dept_Fix[i]['Closed'] for i in Dept_Fix})

#SolveModel(model, 'bonmin')

In [93]:
model.Closure.pprint()

Closure : Size=20, Index=Closure_index, Domain=Any, Default=None, Mutable=False
    Key   : Value
    ALISA :     0
    BEAMR :     0
    BEATY :     0
    CHASZ :     0
    CREAK :     0
    EARTZ :     0
    EVANS :     0
    KNAVE :     0
    NELYN :     0
    NEXAR :     0
    NIKKY :     0
    NOBLR :     0
    PHILS :     0
    SMITH :     0
    TREAT :     0
    WHALT :     0
    WICKR :     1
    WILEY :     0
    WORTH :     0
    WYMON :     0


In [ ]:
# Creates the model and set solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# Flight i uses dept fix j at time k
x = {(i,j):model.NewIVar(f"x_{i}_{j}") \
     for i in Flights.keys() for j in Dept_Fix.keys()}
#status = solver.Solve(model)

# Minimize throughput

print(f"Status = {solver.StatusName(status)}")